In [0]:
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "AK")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "SK")
patient_df = spark.read.csv("s3://my-bucket-om/capstone/Patient_records.csv", header=True, inferSchema=True)
hospital_df = spark.read.csv("s3://my-bucket-om/capstone/hospital.csv", header = True, inferSchema = True)
patient_df.show()
hospital_df.show()
patient_df.printSchema()
hospital_df.printSchema()

patient_df.createOrReplaceTempView("patient_df")
hospital_df.createOrReplaceTempView("hospital_df")


+----------+------------+--------------+------------------+--------------+----------------+--------------------+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date| patient_phone|    disease_name|                city|hospital_id|
+----------+------------+--------------+------------------+--------------+----------------+--------------------+-----------+
|    187158|      Harbir|        Female|        1924-06-30|+91 0112009318|    Galactosemia|            Rourkela|      H1001|
|    112766|    Brahmdev|        Female|        1948-12-20|+91 1727749552|  Bladder cancer|        Tiruvottiyur|      H1016|
|    199252|     Ujjawal|          Male|        1980-04-16|+91 8547451606|   Kidney cancer|           Berhampur|      H1009|
|    133424|     Ballari|        Female|        1969-09-25|+91 0106026841|         Suicide|        Bihar Sharif|      H1017|
|    172579|     Devnath|        Female|        1946-05-01|+91 1868774631|    Food allergy|         Bidhannagar|      H1019|


In [0]:
%sql
SELECT 
    h.Hospital_name,
    h.city,
    COUNT(p.Patient_id) AS Patient_Count
FROM 
    patient_df p
JOIN 
    hospital_df h 
    ON p.hospital_id = h.Hospital_id
GROUP BY 
    h.Hospital_name, h.city
ORDER BY 
    Patient_Count DESC
LIMIT 1;

Hospital_name,city,Patient_Count
Manipal Hospitals,Bengaluru,9


In [0]:
output = spark.sql("""
SELECT h.Hospital_name, h.city, COUNT(p.Patient_id) AS Patient_Count
FROM patient_df p
JOIN hospital_df h ON p.hospital_id = h.Hospital_id
GROUP BY h.Hospital_name, h.city
ORDER BY Patient_Count DESC
LIMIT 1;
""")



output.show()

+-----------------+---------+-------------+
|    Hospital_name|     city|Patient_Count|
+-----------------+---------+-------------+
|Manipal Hospitals|Bengaluru|            9|
+-----------------+---------+-------------+



In [0]:
output.write.format("redshift").option("url", "jdbc:redshift://default-workgroup.9********8.us-east-2.redshift-serverless.amazonaws.com:5439/dev").\
   option("dbtable", "capstone.HospitalServingMaxPatients").\
   option("aws_iam_role", "arn:aws:iam::90******8:role/redshift_admin").\
   option("tempdir", "s3a://glue-bucket-om999/temp2/").\
   option("user", "admin").\
   option("password", "********").save()